# Зависимости

In [43]:
import csv
from functions import (load_portion_functions, append_functions_infile, 
                       filter_functions_name, load_functions)
from settings import FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH

# Выполнение анализа имен функций

`uniq_names` - словарь уникальных значений функций со стчетчиком

In [ ]:
uniq_names = {}

### Подсчет уникальных имен функций

Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональный анализ функций.

* batch - размер порции (1000 для ОЗУ объемом 4 Гб)

In [ ]:
batch = 1000
pos = 0
functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, 1)

print('Progress:')
while True:
    functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, batch)
    if not functions:
        break

    # Упростить названия функций
    for i,f in enumerate(functions):
        functions[i][0]['value'] = functions[i][0]['value'].lower().replace('__', '')
        
    # Добавить имена в словарь подсчета
    for func in functions:
        uniq_names[func[0]['value']] = uniq_names.get(func[0]['value'], 0) + 1
    
    print('.', end='')

In [ ]:
print('Уникальных функций:', len(uniq_names))

### Выбор самых частотных имен функций

* thresholder - задает порог вхождения функции в выборку

In [ ]:
thresholder = 300
names = []
for name, count in uniq_names.items():
    if count > thresholder:
        names.append([name,count])
names = sorted(names, key=lambda x: x[1], reverse=True)

In [ ]:
print('Уникальных имен функций: ', len(names))

### Фильтрация имен функций по значению

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [ ]:
bad_names = ['main', 'run', 'init',
            'iter', 'enter','exit',
            'hash', 'del',
            'nonzero', 'func', 'foo', '_run']
names = list(filter(lambda name: name[0] not in bad_names and len(name[0]) > 2, names))

In [ ]:
print('Уникальных имен функций после чистки на неподходящие имена: ', len(names))

In [ ]:
names[:5]

In [ ]:
# Избавление от информации подсчета
names = [name for name, count in names]

### Фильтрация функций по выбранным именам

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

In [ ]:
pos = 0
functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, 1)

print('Progress:')
while True:
    functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, batch)
    if not functions:
        break
        
    append_functions_infile(
        FILTERED_FUNCTIONS_PATH, 
        filter_functions_name(functions, names)
    )
    print('.', end='')

# Удаление лишних узлов функций

Загрузка функций из файла

In [2]:
functions = load_functions(FILTERED_FUNCTIONS_PATH)

JSONDecodeError


In [3]:
functions[0]

[{'type': 'FunctionDef',
  'children': [191, 196, 240],
  'value': 'get_context_data'},
 {'type': 'arguments', 'children': [192, 194, 195]},
 {'type': 'args', 'children': [193]},
 {'type': 'NameParam', 'value': 'self'},
 {'type': 'defaults'},
 {'type': 'kwarg', 'value': 'kwargs'},
 {'type': 'body', 'children': [197, 207, 220, 230, 238]},
 {'type': 'Assign', 'children': [198, 199]},
 {'type': 'NameStore', 'value': 'context'},
 {'type': 'Call', 'children': [200, 206]},
 {'type': 'AttributeLoad', 'children': [201, 205]},
 {'type': 'Call', 'children': [202, 203, 204]},
 {'type': 'NameLoad', 'value': 'super'},
 {'type': 'NameLoad', 'value': 'UsageView'},
 {'type': 'NameLoad', 'value': 'self'},
 {'type': 'attr', 'value': 'get_context_data'},
 {'type': 'NameLoad', 'value': 'kwargs'},
 {'type': 'Assign', 'children': [208, 217]},
 {'type': 'SubscriptStore', 'children': [209, 215]},
 {'type': 'AttributeLoad', 'children': [210, 214]},
 {'type': 'SubscriptLoad', 'children': [211, 212]},
 {'type': 

#### Извлечение названия функции из функции

Удаление из обучающей выборки названия функции и сохранение в отдельном массиве для дальнейшей валидации.

In [4]:
labels = []
for i, func in enumerate(functions):
    labels.append(func[0]['value'])
    del functions[i][0]

In [5]:
print(labels[0])
functions[0]

get_context_data


[{'type': 'arguments', 'children': [192, 194, 195]},
 {'type': 'args', 'children': [193]},
 {'type': 'NameParam', 'value': 'self'},
 {'type': 'defaults'},
 {'type': 'kwarg', 'value': 'kwargs'},
 {'type': 'body', 'children': [197, 207, 220, 230, 238]},
 {'type': 'Assign', 'children': [198, 199]},
 {'type': 'NameStore', 'value': 'context'},
 {'type': 'Call', 'children': [200, 206]},
 {'type': 'AttributeLoad', 'children': [201, 205]},
 {'type': 'Call', 'children': [202, 203, 204]},
 {'type': 'NameLoad', 'value': 'super'},
 {'type': 'NameLoad', 'value': 'UsageView'},
 {'type': 'NameLoad', 'value': 'self'},
 {'type': 'attr', 'value': 'get_context_data'},
 {'type': 'NameLoad', 'value': 'kwargs'},
 {'type': 'Assign', 'children': [208, 217]},
 {'type': 'SubscriptStore', 'children': [209, 215]},
 {'type': 'AttributeLoad', 'children': [210, 214]},
 {'type': 'SubscriptLoad', 'children': [211, 212]},
 {'type': 'NameLoad', 'value': 'context'},
 {'type': 'Index', 'children': [213]},
 {'type': 'Str',

#### Удаление длинных последовательностей

Некоторые листы или словари могу иметь много элементов. Большие списки и словари будут обрезаться.

Удаление значений словарей

In [6]:
for i,func in enumerate(functions):
    for j,node in enumerate(func):
        if node['type'] == 'Dict':
            try:
                functions[i][j]['children'] = functions[i][j]['children'][:len(functions[i][j]['children'])//2]
            except KeyError:
                pass

In [7]:
uniq_values = {}

In [8]:
for func in functions:
    for node in func:
        try:
            uniq_values[node['value']] = uniq_values.get(node['value'], 0) + 1
        except KeyError:
            pass

In [9]:
print('Уникальных значений:', len(uniq_values))

Уникальных значений: 165907


In [10]:
thresholder = 100
values = []
for value, count in uniq_values.items():
    if count > thresholder:
        values.append([value,count])
values = sorted(values, key=lambda value: value[1], reverse=True)

In [11]:
print('Уникальных значений после отбора:', len(values))

Уникальных значений после отбора: 1296


In [12]:
values[:5]

[['self', 185954],
 ['None', 25139],
 ['result', 23946],
 ['0', 15017],
 ['True', 12940]]

In [13]:
# Избавление от информации подсчета
values = [value for value, count in values]

In [14]:
for i,func in enumerate(functions):
    for j,node in enumerate(func):
        try:
            if node['value'] in values:
                functions[i][j] = node['value']
            else:
                functions[i][j] = node['type']
        except:
            functions[i][j] = node['type']
            continue

In [24]:
len(functions)

54213

In [25]:
for func in functions:
    if type(func) != list:
        print('F')
    for node in func:
        if type(node) != str:
            print('S')

In [27]:
ave = sum(len(func) for func in functions) / len(functions)
print('Средняя длина вектора АСД: ' + str(ave))

Средняя длина вектора АСД: 58.36911810820283


In [33]:
assert len(labels) == len(functions)

In [39]:
functions_dataset = [(label, func) for (label, func) in zip(labels, functions)]

In [42]:
len(functions_dataset)

54213

In [44]:
def funcs_to_file(functions_train):
    with open(f'snip_d_t.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'ast'])

        for func in functions_train:
                writer.writerow({'name':func[0], 'ast':','.join(func[1])})

In [45]:
funcs_to_file(functions_dataset)